In [ ]:
from glob import glob
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize
from tqdm import tqdm

Get the Images and Masks from the filepath

In [ ]:
image_filenames = glob("Path to LIDC-IDRI Preprocessed Image File")
mask_filenames = glob("Path to LIDC-IDRI Preprocessd Mask File")

In [ ]:
# Creating zeros array to saved loaded files after reshaping
image = np.zeros((len(image_filenames),512,512,1))
mask = np.zeros((len(mask_filenames),512,512,1))

In [ ]:
# Loading the npy files and storing them in image array
for n, img in tqdm(enumerate(image_filenames),total=len(image_filenames)):
    img = np.load(img)
    img = np.reshape(img, (512,512,1))
    image[n] = img

100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 391.79it/s]


In [ ]:
# Loading the npy files and storing them in mask array
for n, img in tqdm(enumerate(mask_filenames),total=len(mask_filenames)):
    img = np.load(img)
    img = np.reshape(img, (512,512,1))
    mask[n] = img

100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 892.48it/s]


Train, Test, Validation Split

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(image, mask, test_size=0.2, random_state=42)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

Dice Coefficient Loss Function

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice

def dice_coef_loss(y_true, y_pred):
    loss = 1 - dice_coef(y_true, y_pred)
    return loss


Conv-Unet Model

In [ ]:
def conv_block(inputs, filters, kernel_size=3, activation='relu', padding='same'):
    conv = layers.Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
    conv = layers.BatchNormalization()(conv)
    conv = layers.Conv2D(filters, kernel_size, activation=activation, padding=padding)(conv)
    conv = layers.BatchNormalization()(conv)
    return conv

def convunet_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Contracting Path
    conv1 = conv_block(inputs, 32)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = conv_block(pool1, 64)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = conv_block(pool2, 128)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = conv_block(pool3, 256)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = conv_block(pool4, 512)

    # Expanding Path
    up6 = layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv5)
    up6 = layers.concatenate([up6, conv4])
    conv6 = conv_block(up6, 256)

    up7 = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv6)
    up7 = layers.concatenate([up7, conv3])
    conv7 = conv_block(up7, 128)

    up8 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv7)
    up8 = layers.concatenate([up8, conv2])
    conv8 = conv_block(up8, 64)

    up9 = layers.Conv2DTranspose(32, 2, strides=(2, 2), padding='same')(conv8)
    up9 = layers.concatenate([up9, conv1])
    conv9 = conv_block(up9, 32)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv9)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (512, 512, 1)

model = convunet_model(input_shape)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss=dice_coef_loss, metrics=[dice_coef])

In [ ]:
# Defining checkpoints
checkpoint = ModelCheckpoint('unetmodeltest.h5', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3, verbose=1)
callbacks = [checkpoint, early_stopping, reduce_lr]

# Train the model
history = model.fit(train_x, train_y, validation_data=(val_x, val_y), batch_size=4, epochs=20, callbacks=callbacks)


Epoch 1/20
80/80 [==============================] - ETA: 0s - loss: 0.9934 - dice_coef: 0.0066
Epoch 1: val_loss improved from inf to 0.97838, saving model to newmodelv5.h5
80/80 [==============================] - 44s 412ms/step - loss: 0.9934 - dice_coef: 0.0066 - val_loss: 0.9784 - val_dice_coef: 0.0216 - lr: 0.0010
Epoch 2/20
80/80 [==============================] - ETA: 0s - loss: 0.9881 - dice_coef: 0.0119
Epoch 2: val_loss did not improve from 0.97838
80/80 [==============================] - 33s 419ms/step - loss: 0.9881 - dice_coef: 0.0119 - val_loss: 0.9983 - val_dice_coef: 0.0017 - lr: 0.0010
Epoch 3/20
80/80 [==============================] - ETA: 0s - loss: 0.9819 - dice_coef: 0.0181
Epoch 3: val_loss did not improve from 0.97838
80/80 [==============================] - 34s 427ms/step - loss: 0.9819 - dice_coef: 0.0181 - val_loss: 0.9921 - val_dice_coef: 0.0079 - lr: 0.0010
Epoch 4/20
80/80 [==============================] - ETA: 0s - loss: 0.9757 - dice_coef: 0.0243
Epoch 4

In [ ]:
test_loss, test_dice_coef = model.evaluate(test_x, test_y)
print("Test Loss:", test_loss)
print("Test Dice Coefficient:", test_dice_coef)

ResourceExhaustedError: Graph execution error:

Detected at node 'model/batch_normalization_2/FusedBatchNormV3' defined at (most recent call last):
    File "c:\users\akash\appdata\local\programs\python\python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\users\akash\appdata\local\programs\python\python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\users\akash\appdata\local\programs\python\python39\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\users\akash\appdata\local\programs\python\python39\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\users\akash\appdata\local\programs\python\python39\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "c:\users\akash\appdata\local\programs\python\python39\lib\site-packages\to

In [ ]:
model.save("unetmodel")

INFO:tensorflow:Assets written to: unetmodelnewv2\assets


INFO:tensorflow:Assets written to: unetmodelnewv2\assets
